In [19]:
import affine
import geospatial_tools as gstools
import numpy as np
import os
import pandas as pd
import rasterio

from osgeo import gdal, gdal_array, osr
from pyproj import Proj

In [20]:
def convert_latlon_to_utm(utm_zone, coords):
    """
    convert lat/lon to coordinates of specific utm zone
    
    coordinates should be in (lat, lon) format
    """
    #coords = (41.6604518, -89.4559081)
    p = Proj(proj='utm', ellps='WGS84', zone = utm_zone, south=False)

    return p(coords[1], coords[0])

In [21]:
def CreateGeoTiff(outRaster_name, array, geotransform, epsg):
    
    """
    write a geotiff output using an array
    
    Parameters
    outRaster_name: string
        output name 
    
    array: nd array
        an array that writes to a geotiff file
        
    geotransform: tuple
        geotranform parameter
        GeoTransform[0] /* top left x */
        GeoTransform[1] /* w-e pixel resolution */
        GeoTransform[2] /* 0 */
        GeoTransform[3] /* top left y */
        GeoTransform[4] /* 0 */
        GeoTransform[5] /* n-s pixel resolution (negative value) */
    
    epsg: str
        epsg id  
    
    return
    
    none: 
        write a geotiff file to a specified path
    """
        
    driver = gdal.GetDriverByName('GTiff')
    rows, cols, band = array.shape
    ds = driver.Create(outRaster, cols, rows, band, gdal.GDT_Float64)
    ds.SetGeoTransform(geotransform)
    
    epsg = int(epsg.split(':')[1])
    srs = osr.SpatialReference()            # establish encoding
    srs.ImportFromEPSG(epsg)                # WGS84 lat/long
    ds.SetProjection(srs.ExportToWkt())

    array = np.moveaxis(array, -1, 0)

    for i, image in enumerate(array, 1):
        ds.GetRasterBand(i).WriteArray(image)
    ds = None

In [22]:
def convert_mapcoord_to_pixelcoord(coords, input_raster):
    
    """
    Return floating-point value that corresponds to given point.
    
    Parameters
    coords: tuple
        (lat, lon) or (northing, easting) format
    
    input_raster: str
        path to a raster file
    
    return
    
    pixel_coord: tuple
        pixel coordinate that corresponds to row, column format
    """
    
    x, y = float(coords[0]), float(coords[1])
    
    input_ds = gdal.Open(input_raster, gdal.GA_ReadOnly)
    
    forward_transform =  \
        affine.Affine.from_gdal(*input_ds.GetGeoTransform())
    
    reverse_transform = ~forward_transform
    px, py = reverse_transform * (x, y)
    
    # if pixel location needs to be ajusted to center of a pixel, change the number (px + 0.5)
    px, py = int(px + 0.0), int(py + 0.0)
    pixel_coords = py, px # row and column format

    return pixel_coords

In [23]:
def retrieve_pixel_value_from_array(pixel_coords, input_array, window_size):
    
    """
    Return floating-point value that corresponds to given point.
    
    Parameters
    pixel_coords: tuple
        pixel coordinates in row and column order
    
    input_array: str
        path to a single band raster file 
    
    window_size : int
    
    return
    
    data_array: list
        pixel value that corresponds to lat/lon
    """
    
    py = pixel_coords[0]
    px = pixel_coords[1]
    
    data_array = input_array
    
    if window_size == 1:
        pix_val = round(data_array[py, px], 4)
        
    elif window_size % 2 != 0:
        interval = int((window_size - 1) / 2)
        values = data_array[(py - interval):(py + (interval+1)), (px - interval):(px + (interval+1))]
        pix_val = round(values.mean(), 4)
        
    elif window_size % 2 == 0:
        print ('window size must be an odd number')
        
    else:
        print ('point is located out of range')
    
    return pix_val

In [24]:
def retrieve_pixel_values_from_raster(coords, input_raster, window_size):
    
    """
    extract pixel values from image

    Parameters:
    ----------
    coords : tuple
        lat/lon or easting/northing coords
    input_raster : string
        absolute path 
    window size : int
         a three dimentional array with TOA values of the red band of an image
    
    Returns:
    ------
    pix_vals : list
      list of pixel values for all bands
    """
        
    input_ds = gdal.Open(input_raster, gdal.GA_ReadOnly)
    
    pixel_coords = convert_mapcoord_to_pixelcoord(coords, input_raster)
    
    pix_vals = []
    for band in range(input_ds.RasterCount):
        band += 1
        src_array = np.array(input_ds.GetRasterBand(band).ReadAsArray()).astype(np.float64)
        value = retrieve_pixel_value_from_array(pixel_coords, src_array, window_size)
        pix_vals.append(value)
        
    return pix_vals

In [25]:
# load input csv file into a dataframe
data_dir = '/data'

In [26]:
band_list_pixel = ["b1" , "b2", "b3", "b4", "b5", "b6", "b7",
             "b8", "b8a", "b9", "b10", "b11", "b12"]

In [27]:
s2_path = os.path.join(data_dir, 'sample_s2.tif')
tsavi_path = os.path.join(data_dir, 'sample_tsavi.tif')

### use case: extract pixel values using lat/lon

In [28]:
lat, lon = 39.22174, -87.80272

In [29]:
coords = (lat, lon)

In [30]:
utm_zone = gstools.get_utm_zone(lat, lon)
utm_zone

(16, 'N')

In [31]:
# convert lat/lon coordinates to utm
coords_utm = convert_latlon_to_utm(utm_zone[0], coords)
coords_utm

(430708.0839357439, 4341690.715349512)

#### multi-bands (sentinel2)

In [32]:
# obtain crs (UTM zone if it is projected) from the firat image of a list
with rasterio.open(s2_path) as src:

    # extract pixel values from point location
    pixel_values = retrieve_pixel_values_from_raster(coords_utm, s2_path, 1)

In [33]:
pixel_values

[1175.0,
 1204.0,
 1195.0,
 1159.0,
 1343.0,
 2217.0,
 2630.0,
 2407.0,
 2863.0,
 465.0,
 10.0,
 2217.0,
 1344.0]

### if you want pixel values oupput to a dataframe
#### pix_dict = dict(zip(band_list_pixel, pixel_values))
#### df_pix = pd.DataFrame([pix_dict], columns=pix_dict.keys())

In [34]:
pix_dict = dict(zip(band_list_pixel, pixel_values))
df_pix = pd.DataFrame([pix_dict], columns=pix_dict.keys())
df_pix

,b1,b2,b3,b4,b5,b6,b7,b8,b8a,b9,b10,b11,b12
0,1175.0,1204.0,1195.0,1159.0,1343.0,2217.0,2630.0,2407.0,2863.0,465.0,10.0,2217.0,1344.0


### if you compute index layer on the fly, use "retrieve_pixel_value_from_array" function
#### ndvi_p = retrieve_pixel_value_from_array(pixel_coords, ndvi, 3)

#### single band

In [35]:
# obtain crs (UTM zone if it is projected) from the firat image of a list
with rasterio.open(tsavi_path) as src:

    # extract pixel values from point location
    pixel_value = retrieve_pixel_values_from_raster(coords_utm, tsavi_path, 1)

In [36]:
pixel_value

[0.3013]